In [38]:
import os
import json
import pickle as pk

data_dir = 'cross_validation/subtopics_suggestions'

In [39]:
def load_origin_test(data_dir, idx):
    with open(os.path.join(data_dir, f'test_qids_fold{idx}.pkl'), 'rb') as f:
        return pk.load(f)

def load_origing_train(data_dir, idx):
    data = []
    with open(os.path.join(data_dir, f'train_jsonl_fold{idx}.jsonl'), 'r', encoding='utf8') as f:
        for line in f:
            if len(line) > 0:
                data.append(json.loads(line))
    return data


In [62]:
data_test = load_origin_test(data_dir, 1)
data_train = load_origing_train('cross_validation/subtopics_suggestions', 1)
data_train[0]

{'query': 'french lick resort and casino',
 'pos': ["French Lick Indiana IN hotels Secure Reservations Star Rated Hotels Information and Photos Rates and Availability Amenities Local Attractions Internet Prices! Directions and more... Book online and save! Thank you for Choosing USAHMI.com!! United States > Indiana > French Lick French Lick, Indiana Hotels This site is a great resource for French Lick, IN hotels and much more! If you're looking for information about French Lick travel you've come to the right place. Arrival Date Jan Feb Mar Apr May Jun Jul Au"],
 'neg': ["Elegance Returning To Midwest's Largest Resort / Francine Silverman / Oct 1997 Hotel Online Special Releases Elegance Returning To Midwest's Largest Resort By Francine Silverman The glory days of Indiana's two grandest resorts are definitely gone. When it opened in 1902, West Baden Springs Hotel was dubbed the Eighth Wonder of the World. Closed as a hotel in 1932 and unoccupied since 1983, the crumbling structure is u

In [41]:
def get_qid_results_cv(indri_rank_dict,ranker,test_qids):

    #把 qid ，query，subtopic都读进
    filename="subtopics_suggestions" if not "intent" in load else "subtopics_intents"
    with open(f'{base_path}/diversification_data/{filename}.json','r') as f:
        qid_query_subtopic=json.load(f)
    
    run_list=[]
    score_list=[]
    for qid in tqdm(test_qids):

        if qid=='95' or qid=='100':
            continue

        prompt_former="Consider a query A and a passage B. The subtopics of this query are as follows:"
        prompt_latter="These subtopics are specific and subdivided topics related to the query. Please determine whether passage B contains possible intents related to query A, and give the answer by predicting 'yes' or 'no'"

        subtopics=qid_query_subtopic[qid]['subtopic'].split(",")

        prompt=""+prompt_former
        for index,subtopic in enumerate(subtopics):
            subtopic=" ".join(subtopic.split("_"))
            number=index+1
            if number==len(subtopics):
                prompt=prompt+f"{number}.'"+subtopic+"'. "
            else:
                prompt=prompt+f"{number}.'"+subtopic+"', "

        prompt=prompt+prompt_latter

        query,doc_list=indri_rank_dict[qid]
        docids=[item[0] for item in doc_list]
        doc_contents=[item[1][:MAX_LENGTH] for item in doc_list]
        rank_list=[[query,doc] for doc in doc_contents]
        
        if isinstance(ranker,FlagLLMReranker):
            scores=ranker.compute_score(rank_list,batch_size=1,prompt=prompt)
        else:
            scores=ranker.compute_score(rank_list,batch_size=32)
        #import pdb;pdb.set_trace()
        
        for i in range(len(docids)):
            docid=docids[i]
            score=scores[i]
            eval_item=ScoredDoc(qid,docid,score)
            score_list.append((qid,docid,score))
            run_list.append(eval_item)
    dict_result=pyndeval.ndeval(qrels, run_list, measures=["alpha-nDCG@20"])
    return dict_result,score_list

def read_pickle(path):
    with open(path,'rb') as f:
        data=pk.load(f)
    return data

In [28]:
data_test # f'test_qids_fold{idx}.pkl'

['5',
 '6',
 '8',
 '13',
 '17',
 '19',
 '23',
 '34',
 '38',
 '45',
 '46',
 '56',
 '61',
 '62',
 '75',
 '81',
 '84',
 '99',
 '107',
 '108',
 '112',
 '123',
 '124',
 '126',
 '130',
 '131',
 '146',
 '147',
 '151',
 '153',
 '155',
 '160',
 '171',
 '178',
 '180',
 '181',
 '183',
 '184',
 '186',
 '190']

In [63]:
indri_rank_dict = read_pickle('./indri_data_installed.pkl')
# results_dict,scores=get_qid_results_cv(indri_rank_dict,ranker,test_qids)
# eval_metrics=[results_dict[qid]['alpha-nDCG@20'] for qid in results_dict]
# eval_metrics_overall+=eval_metrics
indri_rank_dict

In [65]:
indri_rank_dict['1']

['obama family tree',
 [['clueweb09-en0010-57-32937',
   "Sun-Times News Group :: Obama Family Tree :: Select a STNG publication or site Chicago Sun-Times The Beacon News The Courier News The SouthtownStar The Herald News Lake County News-Sun The Naperville Sun Post-Tribune YourSeason.com Search Chicago Home Autos Homes Jobs Neighborhood Circle Home Deerfield Glencoe Glenview Grayslake Highland Park Lake Forest Libertyville Montgomery Mundelein Northbrook Oswego Vernon Hills Wilmette Winnetka Yorkville Centerstage Roger Ebert PioneerLocal.com Algonquin Countryside Antioch Review Arlington Heights Post Barrington Courier-Review Buffalo Grove Countryside Cary-Grove Countryside The Doings Clarendon Hills Edition The Doings Elmhurst Edition The Doings Hinsdale Edition The Doings La Grange Edition The Doings Oak Brook Edition The Doings Weekly Edition The Doings Western Springs Edition Deerfield Review Des Plaines Times Edison-Norwood Times-Review Elk Grove Times Elm Leaves Evanston Review 

In [44]:
indri_rank_dict.keys()

dict_keys(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '150', '151', '152', '153', '154', '155', '156', '157'

In [48]:
query, docs = indri_rank_dict['1']

In [49]:
query

'obama family tree'

In [61]:
[item[0] for item in docs]

['clueweb09-en0010-57-32937',
 'clueweb09-en0011-02-12744',
 'clueweb09-en0010-57-32259',
 'clueweb09-en0001-02-21397',
 'clueweb09-en0008-45-29117',
 'clueweb09-en0080-22-19802',
 'clueweb09-en0001-02-21241',
 'clueweb09-en0023-08-38154',
 'clueweb09-en0004-34-30809',
 'clueweb09-en0004-34-30810',
 'clueweb09-en0011-72-34006',
 'clueweb09-en0007-63-02101',
 'clueweb09-en0009-30-02655',
 'clueweb09-en0023-08-37688',
 'clueweb09-en0009-30-02620',
 'clueweb09-en0009-30-02580',
 'clueweb09-en0009-30-02702',
 'clueweb09-en0009-30-02633',
 'clueweb09-en0009-30-02436',
 'clueweb09-en0009-30-02476',
 'clueweb09-en0023-08-37892',
 'clueweb09-en0009-30-02567',
 'clueweb09-en0009-30-02830',
 'clueweb09-en0009-30-02648',
 'clueweb09-en0023-08-37942',
 'clueweb09-en0023-08-38419',
 'clueweb09-en0009-30-02438',
 'clueweb09-en0009-30-02785',
 'clueweb09-en0009-30-02629',
 'clueweb09-en0009-30-02565',
 'clueweb09-en0009-30-02741',
 'clueweb09-en0009-30-02737',
 'clueweb09-en0009-30-02586',
 'clueweb0

In [ ]:
[item[0] for item in doc_list]